In [5]:
# Needed if running on Colab
!pip3 install open-spiel
!pip3 install torch
!pip3 install tqdm



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
from go_search_problem import GoProblem, GoState
from heuristic_go_problems import GoProblemLearnedHeuristic, GoProblemSimpleHeuristic
from agents import GreedyAgent, RandomAgent, MCTSAgent, GameAgent, AlphaBetaAgent, IterativeDeepeningAgent
import matplotlib.pyplot as plt
from go_utils import create_go_game
from tqdm import tqdm
from game_runner import run_many
import pickle
from sklearn.model_selection import train_test_split
torch.set_default_tensor_type(torch.FloatTensor)

ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default
/home/codespace/.local/lib/python3.12/site-packages/torch/__init__.py:955: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_t

In [13]:
state = GoState(create_go_game(5))
state.apply_action(1)
print(state.internal_state.move_number())

1


In [11]:
def load_dataset(path: str):
    with open(path, 'rb') as f:
        dataset = pickle.load(f)
    return dataset

dataset_5x5 = load_dataset('dataset_5x5.pkl')
# dataset_9x9 = load_dataset('9x9_dataset.pkl')

In [12]:
def save_model(path: str, model):
    """
    Save model to a file
    Input:
        path: path to save model to
        model: Pytorch model to save
    """
    torch.save({
        'model_state_dict': model.state_dict(),
    }, path)

def load_model(path: str, model):
    """
    Load model from file

    Note: you still need to provide a model (with the same architecture as the saved model))

    Input:
        path: path to load model from
        model: Pytorch model to load
    Output:
        model: Pytorch model loaded from file
    """
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    return model

# Task 1: Convert GameState to Features

In [13]:
def get_features(game_state: GoState):
    """
    Map a game state to a list of features.

    Some useful functions from game_state include:
        game_state.size: size of the board
        get_pieces_coordinates(player_index): get coordinates of all pieces of a player (0 or 1)
        get_pieces_array(player_index): get a 2D array of pieces of a player (0 or 1)
        
        get_board(): get a 2D array of the board with 4 channels (player 0, player 1, empty, and player to move). 4 channels means the array will be of size 4 x n x n
    
        Descriptions of these methods can be found in the GoState

    Input:
        game_state: GoState to encode into a fixed size list of features
    Output:
        features: list of features
    """
    board_size = game_state.size
    
    # TODO: Encode game_state into a list of features
    features = []
    #Get the current state
    #get board
    #
    board = game_state.get_board()
    
    player_0 = list(board[0].flatten())
    player_1 = list(board[1].flatten())
    empty = list(board[2].flatten())
    
    player_to_move = (board[3][0][0])
   
    #could consider adding extra features
    features = player_0+ player_1 + empty
    features.append(player_to_move)

    player_0_count = sum(player_0)
    features.append(player_0_count)
    player_1_count = sum(player_1)
    features.append(player_1_count)
    empty_count = sum(empty)
    features.append(empty_count)

    if player_to_move == 0:
        count_diff = player_0_count - player_1_count
    else:
        count_diff = player_1_count - player_0_count
    features.append(count_diff)

    return features

In [14]:
# Print information about first data point
data_point = dataset_5x5[0]

features =(get_features(data_point[0]))

action = data_point[1]
result = data_point[2]
print(data_point[0])
print("features", features)
print("Action #", action)
print("Game Result", result)
print("features len" , len(features))

GoState(komi=0.5, to_play=B, history.size()=0)

 5 +++++
 4 +++++
 3 +++++
 2 +++++
 1 +++++
   ABCDE

features [np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(1.0), np.

# Task 2: Supervised Learning of a Value Network

In [15]:
class ValueNetwork(nn.Module):
    def __init__(self, input_size):
      super(ValueNetwork, self).__init__()

      # TODO: What should the output size of a Value function be?
      output_size = 1

      # TODO: Add more layers, non-linear functions, etc.
      #self.linear = nn.Linear(input_size, output_size)
      self.in_layer = nn.Linear(input_size,100)
        
      self.layer_1 = nn.Linear(100,90)

      self.layer_2 = nn.Linear(90,70)

      self.layer_3 = nn.Linear(70,50)

      self.out_layer = nn.Linear(50,1)

      self.sigmoid = nn.Sigmoid()
      self.tanh = nn.Tanh()
      self.relu = nn.ReLU()

    def forward(self, x):
      """
      Run forward pass of network

      Input:
        x: input to network
      Output:
        output of network
      """
      # TODO: Update as more layers are added
      
      a = self.in_layer(x)
      b = self.tanh(a)

      c = self.layer_1(b)
      d = self.relu(c)

      e = self.layer_2(d)
      f = self.relu(e)

      g = self.layer_3(f)
      h = self.relu(g)

      i = self.out_layer(h)
      j = self.sigmoid(i)
      return (j)

In [16]:
# This will not produce meaningful outputs until trained, but you can test for syntax errors

features_tensor = torch.Tensor(features)

value_net = ValueNetwork(len(features))
print("predicted Value", value_net(features_tensor))

predicted Value tensor([0.5140], grad_fn=<SigmoidBackward0>)


In [9]:
def train_value_network(dataset, num_epochs, learning_rate):
    """
    Train a value network on the provided dataset.

    Input:
        dataset: list of (state, action, result) tuples
        num_epochs: number of epochs to train for
        learning_rate: learning rate for gradient descent
    Output:
        model: trained model
    """
    # Make sure dataset is shuffled for better performance
    random.shuffle(dataset)

    train_data, test_data = train_test_split(dataset, test_size=0.2)

        # You may find it useful to create train/test sets to better track performance/overfit/underfit
    # TODO: Create model
    #Make sure this size is right
    model = ValueNetwork(len(get_features(GoState(create_go_game(5)))))

    # TODO: Specify Loss Function
    loss_function = nn.MSELoss()

    # You can use Adam, which is stochastic gradient descent with ADAptive Momentum
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    batch_size = 32
    batch_loss = 0.0
    batch_counter = 0 
   
    for epoch in range(num_epochs):
        test_loss = 0
        for data_point in train_data:
            state = data_point[0]
            features = get_features(state)
            features_tensor = torch.tensor(features,dtype=torch.float32)

            # TODO: What should the desired output of the value network be?
            # Note: You will have to convert the label to a torch tensor to use with torch's loss functions
            label = torch.tensor(data_point[2],dtype=torch.float32)

            # TODO: Get model prediction of value
            prediction = model.forward(features_tensor) 

            # TODO: Compute Loss for data point
            loss = loss_function(prediction,label)
            
            batch_loss += loss
            batch_counter += 1
            if batch_counter % batch_size == 0:
                # Call backward to run backward pass and compute gradients
                batch_loss.backward()

                # Run gradient descent step with optimizer
                optimizer.step()

                # Reset gradient for next batch
                optimizer.zero_grad()
                batch_loss = 0.0
                batch_counter = 0 
            
        for data_point in test_data:
            state = data_point[0]
            features = get_features(state)
            features_tensor = torch.tensor(features, dtype=torch.float32)

            label = torch.tensor(data_point[1])
            prediction = model.forward(features_tensor)

            test_loss += loss_function(prediction, label).item()

        avg_test_loss = test_loss / len(test_data)
        print("average_test lossin in epoch " + str(epoch) + " is ", avg_test_loss)

    return model

value_model = train_value_network(dataset_5x5, 10, 1e-5)
save_model("value_model.pt", value_model)

/Users/matiasbronner/Desktop/cs410/cs410_env/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


average_test lossin in epoch 0 is  244.89709839906098
average_test lossin in epoch 1 is  245.89332103780202
average_test lossin in epoch 2 is  245.0771660546067
average_test lossin in epoch 3 is  244.6143265086181
average_test lossin in epoch 4 is  244.39116888035804
average_test lossin in epoch 5 is  244.25442959156493
average_test lossin in epoch 6 is  244.1481239174706
average_test lossin in epoch 7 is  244.0511907615367
average_test lossin in epoch 8 is  243.974725548273
average_test lossin in epoch 9 is  243.91335612646353


## Comparing Learned Value function against other Agents

In [ ]:
class GoProblemLearnedHeuristic(GoProblem):
    def __init__(self, model=None, state=None):
        super().__init__(state=state)
        self.model = model
        
    def __call__(self, model=None):
        """
        Use the model to compute a heuristic value for a given state.
        """
        return self

    def encoding(self, state):
        """
        Get encoding of state (convert state to features)
        Note, this may call get_features() from Task 1. 

        Input:
            state: GoState to encode into a fixed size list of features
        Output:
            features: list of features
        """
        # TODO: get encoding of state (convert state to features)
        features = get_features(state)
        features_tensor = torch.tensor(features, dtype=torch.float32)
        return features_tensor

    def heuristic(self, state, player_index):
        """
        Return heuristic (value) of current state

        Input:
            state: GoState to encode into a fixed size list of features
            player_index: index of player to evaluate heuristic for
        Output:
            value: heuristic (value) of current state
        """
        # TODO: Compute heuristic (value) of current state
        value = 0

        features = self.encoding(state)
        value = self.model.forward(features)
        # Note, your agent may perform better if you force it not to pass
        # (i.e., don't select action #25 on a 5x5 board unless necessary)
        return value

    def __str__(self) -> str:
        return "Learned Heuristic"


def create_value_agent_from_model(value_net):
    """
    Create agent object from saved model. This (or other methods like this) will be how your agents will be created in gradescope and in the final tournament.
    """

    model_path = "value_model.pt"
    # TODO: Update number of features for your own encoding size
    feature_size = len(get_features(GoState(create_go_game(5))))
    model = load_model(model_path, ValueNetwork(feature_size))
    heuristic_search_problem = GoProblemLearnedHeuristic(model)

    # TODO: Try with other heuristic agents (IDS/AB/Minimax)
    learned_agent = GreedyAgent(heuristic_search_problem)
    # learned_agent = RandomAgent()

    return learned_agent

learned_agent = create_value_agent_from_model(value_net)
agent2 = IterativeDeepeningAgent(1)
print("Greedy Agent", agent2)
print("Learned Agent", learned_agent)

run_many(learned_agent, agent2,10)

Greedy Agent IterativeDeepneing + Simple Heuristic
Learned Agent GreedyAgent + Learned Heuristic


  0%|          | 0/5 [00:00<?, ?it/s]

self.moves 0
self.moves 14


 20%|██        | 1/5 [00:33<02:15, 33.99s/it]

self.moves 18
self.moves 11


...

self.moves 18
self.moves 23


100%|██████████| 5/5 [03:10<00:00, 38.18s/it]

Agent 1: GreedyAgent + Learned Heuristic Score: 10.0
Agent 2: IterativeDeepneing + Simple Heuristic Score: -10.0
Agent 1: GreedyAgent + Learned Heuristic Score with Black (first move): 5.0
Agent 2: IterativeDeepneing + Simple Heuristic Score with Black (first move): -5.0
Agent 1: GreedyAgent + Learned Heuristic Average Duration: 0.001136414081183912
Agent 2: IterativeDeepneing + Simple Heuristic Average Duration: 0.998583859134767
Agent 1: GreedyAgent + Learned Heuristic Longest Duration: 0.003401041030883789
Agent 2: IterativeDeepneing + Simple Heuristic Longest Duration: 1.0001869201660156
Agent 1: GreedyAgent + Learned Heuristic Average Time Remaining: 33.879002118110655
Agent 2: IterativeDeepneing + Simple Heuristic Average Time Remaining: 15.029584789276123
Agent 1: GreedyAgent + Learned Heuristic Min Time Remaining: 25.98527193069458
Agent 2: IterativeDeepneing + Simple Heuristic Min Time Remaining: 14.999572038650513


(10.0, -10.0)

: 

# Task 3: Supervised Learning of a Policy Network

In [11]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, board_size=5):
      super(PolicyNetwork, self).__init__()
    

     

      # TODO: Add more layers, non-linear functions, etc.
      #self.linear = nn.Linear(input_size, output_size)
      self.in_layer = nn.Linear(input_size,128)
        
      self.layer_1 = nn.Linear(128,50)

      self.layer_2 = nn.Linear(50,32)

      self.layer_3 = nn.Linear(32,26)

      self.relu = nn.ReLU()

    def forward(self, x):
      """
      Run forward pass of network

      Input:
        x: input to network
      Output:
        output of network
      """
      # TODO: Update as more layers are added
      
      a = self.in_layer(x)
      b = self.relu(a)

      c = self.layer_1(b)
      d = self.relu(c)

      e = self.layer_2(d)
      f = self.relu(e)

      g = self.layer_3(f)
     
      return g

In [71]:
# This will not produce meaningful outputs until trained, but you can test for syntax errors
features_tensor = torch.Tensor(features)
policy_net = PolicyNetwork(len(features))
print("Predicted Action Probabilities", policy_net(features_tensor))

Predicted Action Probabilities tensor([ 0.0081, -0.0538,  0.0305,  0.0924,  0.1959, -0.0186, -0.0499, -0.0316,
        -0.0877,  0.1153,  0.1518, -0.0040,  0.0746,  0.0237, -0.3385, -0.0298,
         0.0241,  0.0460,  0.0482,  0.0562,  0.2286, -0.0640, -0.0027,  0.0121,
        -0.1015,  0.1124], grad_fn=<ViewBackward0>)


In [ ]:
def train_policy_network(dataset, num_epochs, learning_rate):
    """
    Train a policy network on the provided dataset.

    Input:
        dataset: list of (state, action, result) tuples
        num_epochs: number of epochs to train for
        learning_rate: learning rate for gradient descent
    Output:
        model: trained model
    """
    random.shuffle(dataset)
    train_data, test_data = train_test_split(dataset, test_size=test_size, random_state=42)

    model = PolicyNetwork(len(get_features(GoState(create_go_game(5)))))

    loss_function = nn.CrossEntropyLoss()

    # You can use Adam, which is stochastic gradient descent with ADAptive Momentum
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        for data_point in dataset:
          
            state = data_point[0]
            features = get_features(state)
            features_tensor = torch.tensor(features,dtype = torch.float32)

            # TODO: What should the desired output of the value network be?
            # Note: You will have to convert the label to a torch tensor to use with torch's loss functions
            label = torch.tensor(data_point[1])
            # TODO: Get model estimate of value
            prediction = model.forward(features_tensor)

            # TODO: Compute Loss for data point
            loss = loss_function(prediction,label)

            # Call backward to run backward pass and compute gradients
            loss.backward()
        
            # Run gradient descent step with optimizer
            optimizer.step()
            optimizer.zero_grad()
            
    return model

policy_net = train_policy_network(dataset_5x5, 10, 1e-4)
save_model("policy_model.pt", policy_net)

## Comparing Learned Policy against other Agents

In [ ]:
class PolicyAgent(GameAgent):
    def __init__(self, search_problem, model_path, board_size=5):
        super().__init__()
        self.search_problem = search_problem
        self.model = load_model(model_path, PolicyNetwork(len(get_features(GoState(create_go_game(5)))),5))
        self.board_size = board_size

    def encoding(self, state):
        # TODO: get encoding of state (convert state to features)
        features = get_features(state)
        features_tensor = torch.tensor(features, dtype=torch.float32)
        return features_tensor

    def get_move(self, game_state, time_limit=1):
      """
      Get best action for current state using self.model

      Input:
        game_state: current state of the game
        time_limit: time limit for search (This won't be used in this agent)
      Output:
        action: best action to take
      """

      # TODO: Select LEGAL Best Action predicted by model
      # The top prediction of your model may not be a legal move!
      features = self.encoding(game_state)
      actions = self.model(features)
      actions = torch.argsort(actions,descending=True)
      #check to see if the action is a legal action at the current stateac
      
      for action in actions:
          if(action.item() in self.search_problem.get_available_actions(game_state)):
              if(action.item() != self.board_size**2 ):
                return action.item()
          
      return game_state.size**2
      # Note, you may want to force your policy not to pass their turn unless necessary
      #assert action in self.search_problem.get_available_actions(game_state)
      
    
    def __str__(self) -> str:
        return "Policy Agent"
    
def create_policy_agent_from_model():
    """
    Create agent object from saved model. This (or other methods like this) will be how your agents will be created in gradescope and in the final tournament.    
    """

    model_path = "policy_model.pt"
    agent = PolicyAgent(GoProblem(size=5), model_path)
    return agent

In [92]:
policy_agent = PolicyAgent(GoProblem(size=5),"policy_model.pt")
learned_agent = create_value_agent_from_model(value_net)
print("Policy Agent", policy_agent)
run_many(policy_agent,GreedyAgent(), 100)

Policy Agent Policy Agent


100%|██████████| 50/50 [00:00<00:00, 79.68it/s]

Agent 1: Policy Agent Score: -35.0
Agent 2: GreedyAgent + Simple Heuristic Score: 35.0
Agent 1: Policy Agent Score with Black (first move): -29.0
Agent 2: GreedyAgent + Simple Heuristic Score with Black (first move): 6.0
Agent 1: Policy Agent Average Duration: 8.201638857523602e-05
Agent 2: GreedyAgent + Simple Heuristic Average Duration: 0.0001570437590281169
Agent 1: Policy Agent Longest Duration: 0.0008039474487304688
Agent 2: GreedyAgent + Simple Heuristic Longest Duration: 0.0005230903625488281
Agent 1: Policy Agent Average Time Remaining: 39.86796121835709
Agent 2: GreedyAgent + Simple Heuristic Average Time Remaining: 39.86610211610796
Agent 1: Policy Agent Min Time Remaining: 26.998926639556885
Agent 2: GreedyAgent + Simple Heuristic Min Time Remaining: 26.997395992279053


(-35.0, 35.0)

# Submitting

After you've completed all the tasks in this notebook, you'll want to add your agents to your agents.py file. You'll want to copy the necessary function and class definitions for PolicyAgent, GoProblemLearnedHeuristic, PolicyNetwork, ValueNetwork, and any other methods you referenced. Your agents will ultimately be tested on gradescope by calling create_value_agent_from_model or by create_policy_agent_from_model.